In [2]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import numpy as np
import pandas as pd
import geopandas as gpd
import morefish as mf
import tifffile

Experiment results interested in creating the new model

In [3]:
dirns = ['/gale/ddn/bican/processing/202303021147_HumanA46RepBICANPilot_VMSC01101/output/region_0',
         '/gale/ddn/bican/processing/202302171232_HumanBnstGPeBICANPilot_VMSC01101/output/region_0',
         '/gale/ddn/bican/processing/202210061152_HumanSIGPeWei_VMSC01101/output/region_0',
         '/gale/ddn/bican/processing/202304241136_HumanV1CRep2BICANPilot_VMSC00501/output/region_0']

The following code will check each experiment, and sample tiles which likely to have some cells. 

Complete empty tiles will not be selected.

In each experiment, `n_tile_samples` number of tiles will be sampled for cellpose training

The results will rgb tiff files in the "tifs/" subfolder.

In [6]:
!mkdir tifs -p

In [4]:
n_tile_samples = 10

In [ ]:
for dirn in dirns[4:]:
    *_,name,_,reg = dirn.split('/')
    name = name.split('_')[1]

    mr = mf.MerfishRegion(dirn)
    stats = []
    for tile_i in range(len(mr.tiles.tiles)):
        for z in range(7):
            img = mr.get_tile_image('DAPI',z,tile_i)
            stats.append(np.quantile(img,np.linspace(0,1,11)).tolist()+[z, tile_i])

    stats = pd.DataFrame(stats, columns=[f'q{i}0' for i in range(11)]+['z','tile_i'])

    a=0.4
    for _, (*_,z,tile_i) in stats[stats['q90']>=stats['q90'].quantile(a)].sample(n_tile_samples).iterrows():
        z = int(z)
        tile_i = int(tile_i)
        figfn = f'tifs/{name}_{reg}_z{z}_t{tile_i}-DAPI+PolyT.tif'
        img = mr.get_tile_image_rgb(['DAPI','PolyT'],z,tile_i).astype(np.uint16)
        tifffile.imwrite(figfn, img, dtype=np.uint16)


After tiff file generation, we could load them with cellpose gui for "human-in-the-loop" training.

In the seting of code above, we should select channel 1 as Red and channel 2 as Green in the cellpose GUI